In [0]:
%%capture
!curl -o scripts.py https://raw.githubusercontent.com/markusbkoch/cadCAD/google-colab/tutorials/robot-marbles-part-1/scripts.py
!pip install cadCAD

In [0]:
#@title How many marbles in box A? Then press Ctrl+F9 (or Cmd+F9) to run.
marbles = 10 #@param {type:"number"}
%matplotlib inline
from scripts import run
run(marbles)